# Mixture of experts

In [ ]:
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import pandas as pd
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from smt.applications import MOE
from smt.surrogate_models import QP, RMTB, RMTC

## Data

In [ ]:
PATH_PROC_DATA = os.path.join('data', 'processed')
syn_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_synthetic.csv'))
true_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'))

features = ['d [mm]', 'f [GHz]', 'psPDtot_1 [W/m2]', 'psPDtot_4 [W/m2]']
target = 'pDeltaT * 100 [°C]'
X = syn_data[features]
y = syn_data[target] / 100

X_train, X_valid, y_train, y_valid = train_test_split(X.to_numpy(),
                                                      y.to_numpy()[:, np.newaxis],
                                                      test_size=.2)
X_test = true_data[features].to_numpy()
y_test = true_data[target].to_numpy() / 100

## Quadratic polynimials

In [ ]:
qp = QP()
qp.set_training_values(X_train, y_train)
qp.train()

In [ ]:
y_pred = qp.predict_values(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

y_resid = (y_test - y_pred.ravel())
ae = np.abs(y_resid)
mae = np.mean(ae)
print(f'MAE: {mae}')

## Mixture of experts

In [ ]:
def train(save=False):
    experts = ['QP', 'RMTB', 'RMTC']
    moe = MOE(n_clusters=2,
              smooth_recombination=True,
              allow=experts)
    moe.set_training_values(X_train, y_train)
    moe.train()
    return moe

In [ ]:
force_train = False
save = False

if force_train:
    print('Fitting the surrogate model...')
    moe = train(save)
else:
    try:
        print('Trying to restore the surrogate model...')
        with open(os.path.join('models', '04_mixture_of_experts.pkl'), 'rb') as f:
            moe = pickle.load(f)
        print('Restoring successful.')
    except Exception as e:
        print(e)
        print('Restoring failed. Fitting the surrogate model...')
        moe = train(save)
if save:
    with open(os.path.join('models', '04_mixture_of_experts.pkl'), 'wb') as f:
        pickle.dump(moe, f)

## Evaluating mixture of experts predicting power

In [ ]:
y_pred = moe.predict_values(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

y_resid = (y_test - y_pred.ravel())
ae = np.abs(y_resid)
mae = np.mean(ae)
print(f'MAE: {mae}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))
ax = sns.histplot(x=ae, stat='density', kde=True, ax=ax)
ax.vlines(mae, *ax.get_ybound(), ls='--', label='mean absolute error')
ax.set(xlabel='absolute error (°C)', ylabel='probability density')
ax.legend();

In [ ]:
PATH_ERROR_DATA = os.path.join('data', 'models')
error_data = os.path.join(PATH_ERROR_DATA, 'mixture_of_experts.npy')
save = False
if save:
    with open(error_data, 'wb') as f:
        np.save(f, ae)